In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_cross = pd.read_csv('/content/drive/My Drive/BT4222/data/df_cross_txt.csv')

In [4]:
df_cross.head()

,reviewer_id,preference,listing_id,host_is_superhost,latitude,longitude,price,number_of_reviews_ltm,calculated_host_listings_count,nearby_sites_rating,nearest_station_dist,host_response_time_cross_rate,item_factor_0,user_factor_0,item_factor_1,user_factor_1,item_factor_2,user_factor_2,item_factor_3,user_factor_3,item_factor_4,user_factor_4,item_factor_5,user_factor_5,item_factor_6,user_factor_6,item_factor_7,user_factor_7,item_factor_8,user_factor_8,item_factor_9,user_factor_9,dtld_descr
0,8557223,0.916301,49091,0,1.44255,103.79580,79.0,0,2,0.995731,0.617762,2.0,-0.080090,-0.045423,-0.072907,0.119669,0.077274,0.045258,0.037663,0.085235,-0.038879,0.009194,0.038809,-0.038233,0.030629,-0.072737,0.032604,0.082893,0.287333,-0.098187,0.089866,0.021846,<b>The space</b><br />This is Room No. 2.(avai...
1,1356099,0.919923,50646,0,1.33235,103.78521,80.0,0,1,1.269606,0.399696,0.0,0.081554,-0.049855,-0.105953,0.044933,-0.116901,0.131527,-0.017028,0.111009,-0.086785,-0.002848,-0.020847,0.074678,-0.037473,-0.052219,-0.075626,-0.012367,0.034406,0.001945,0.085947,0.008726,Fully furnished bedroom with a nice view on th...
2,15222393,0.938573,50646,0,1.33235,103.78521,80.0,0,1,1.269606,0.399696,0.0,0.081554,-0.045758,-0.105953,-0.057649,-0.116901,-0.014033,-0.017028,0.041884,-0.086785,0.181634,-0.020847,-0.046492,-0.037473,-0.133720,-0.075626,-0.118024,0.034406,0.067946,0.085947,-0.158080,Fully furnished bedroom with a nice view on th...
3,5543172,0.926675,50646,0,1.33235,103.78521,80.0,0,1,1.269606,0.399696,0.0,0.081554,0.028973,-0.105953,0.145273,-0.116901,0.066612,-0.017028,-0.063751,-0.086785,0.176873,-0.020847,-0.142651,-0.037473,0.024285,-0.075626,0.007321,0.034406,0.025454,0.085947,0.055499,Fully furnished bedroom with a nice view on th...
4,817532,0.910008,50646,0,1.33235,103.78521,80.0,0,1,1.269606,0.399696,0.0,0.081554,-0.055366,-0.105953,-0.037461,-0.116901,0.030717,-0.017028,0.059124,-0.086785,0.139225,-0.020847,0.046353,-0.037473,-0.032285,-0.075626,-0.020036,0.034406,0.008693,0.085947,-0.050897,Fully furnished bedroom with a nice view on th...


In [5]:
X = df_cross['dtld_descr']
y = df_cross['preference']

Data Preprocessing

In [6]:
# clean the dataset by removing special characters
import re 
def clean_str(string):
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)  
    string = re.sub(r"<b>", "", string) 
    string = re.sub(r"</b>", "", string)
    string = re.sub(r"<br />", "", string)
    return string.strip().lower()


In [7]:
X = X.apply(lambda x: clean_str(x))

In [8]:
X[0]

'the spacethis is room no. 2.(available after jan 15 2014) usd750 per month. you may want to take a look at room number 1 as well.room no 1 is available immediately at usd700 per monthits at the far right corner of your screen below.please note that currency is in usd unless stated as sgd.only long term stay (6 months minimum)place is available only to adults.good location.10 minutes walk to train station.10 minutes walk to bus stops. 7 minutes walk to 24 hour food center. 7 minutes walk to two 24 hour supermarkets. 10 minutes walk to 24/7 mcdonalds.30 minutes train ride to orchard road.on fridays/saturdays, you can take the night rider bus from orchard or from clarke quay direct back home and save taxi fare! room is clean,cozy and comfortable.one queensize bed.there is aircondition and fan.washing machine is also available with no extra chaelevator  cable tv  washer  wifi  tv  air conditioning'

Processing the preference

In [13]:
# !pip install tensorflow==1.15
# !pip install "tensorflow_hub>=0.6.0"
# !pip3 install tensorflow_text==1.15

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

In [11]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

In [12]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  sentences_embeddings = session.run(embed(X.tolist()))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [14]:
np.array(sentences_embeddings)

array([[ 0.02836199,  0.04817889,  0.0237379 , ..., -0.05541828,
        -0.03630639, -0.03260371],
       [ 0.06161199,  0.00506089,  0.00593533, ..., -0.03692728,
        -0.04851945,  0.02410396],
       [ 0.06161199,  0.00506089,  0.00593533, ..., -0.03692728,
        -0.04851945,  0.02410396],
       ...,
       [-0.00420356, -0.00637646,  0.02244581, ...,  0.00530929,
        -0.04831173,  0.03966377],
       [ 0.03058015, -0.00673687,  0.01491317, ..., -0.03633141,
        -0.02127949, -0.04001226],
       [ 0.04673671,  0.00263161,  0.02075313, ...,  0.00107984,
        -0.05161076,  0.03911586]], dtype=float32)

In [15]:
df = pd.DataFrame(np.array(sentences_embeddings))

In [16]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0.028362,0.048179,0.023738,0.054598,0.031764,-0.050831,-0.017383,0.015924,-0.052127,-0.049786,-0.006455,0.038961,-0.056586,0.043548,-0.068984,-0.000530,0.068396,-0.021624,0.028426,0.041494,0.045090,-0.036616,-0.009754,-0.041302,-0.046874,0.000300,0.012849,-0.009962,-0.051899,0.009277,0.032329,-0.060189,-0.046378,0.052770,0.063759,-0.046719,0.010710,-0.049082,-0.016740,-0.057737,...,0.015080,-0.060560,0.044304,0.012890,-0.027689,0.052477,0.051479,-0.068320,-0.048422,0.058402,-0.055617,0.003967,-0.049383,-0.022574,0.062601,0.028154,-0.066375,-0.059863,0.014510,0.045695,0.010616,0.012312,0.043418,-0.067739,0.048400,-0.019342,0.022793,-0.068087,-0.018416,-0.022069,0.038599,0.067944,-0.037068,-0.007591,0.067032,-0.053137,-0.010770,-0.055418,-0.036306,-0.032604
1,0.061612,0.005061,0.005935,0.058571,-0.034820,-0.051958,-0.054881,-0.051389,0.020466,-0.062985,-0.062118,0.025811,-0.012073,0.056732,-0.061061,0.053391,0.044413,0.040475,0.034086,0.061892,0.046909,-0.001616,-0.061952,-0.025852,-0.041427,0.015722,-0.032636,0.012678,-0.029335,-0.041698,0.001134,0.058911,-0.027985,0.063520,0.062227,0.063451,0.008441,-0.053186,0.038237,0.058160,...,0.026464,-0.036157,-0.023513,-0.061082,-0.020623,0.059865,0.060694,0.047033,-0.044069,-0.054903,-0.052173,-0.063486,0.038313,-0.061535,0.036823,0.054980,-0.059736,-0.062665,0.007966,0.008084,0.043164,0.033444,-0.049455,-0.023729,0.022630,-0.039991,0.063206,-0.024769,-0.005954,-0.057347,-0.016031,0.061994,0.044697,0.037051,0.026100,-0.059603,-0.000041,-0.036927,-0.048519,0.024104
2,0.061612,0.005061,0.005935,0.058571,-0.034820,-0.051958,-0.054881,-0.051389,0.020466,-0.062985,-0.062118,0.025811,-0.012073,0.056732,-0.061061,0.053391,0.044413,0.040475,0.034086,0.061892,0.046909,-0.001616,-0.061952,-0.025852,-0.041427,0.015722,-0.032636,0.012678,-0.029335,-0.041698,0.001134,0.058911,-0.027985,0.063520,0.062227,0.063451,0.008441,-0.053186,0.038237,0.058160,...,0.026464,-0.036157,-0.023513,-0.061082,-0.020623,0.059865,0.060694,0.047033,-0.044069,-0.054903,-0.052173,-0.063486,0.038313,-0.061535,0.036823,0.054980,-0.059736,-0.062665,0.007966,0.008084,0.043164,0.033444,-0.049455,-0.023729,0.022630,-0.039991,0.063206,-0.024769,-0.005954,-0.057347,-0.016031,0.061994,0.044697,0.037051,0.026100,-0.059603,-0.000041,-0.036927,-0.048519,0.024104
3,0.061612,0.005061,0.005935,0.058571,-0.034820,-0.051958,-0.054881,-0.051389,0.020466,-0.062985,-0.062118,0.025811,-0.012073,0.056732,-0.061061,0.053391,0.044413,0.040475,0.034086,0.061892,0.046909,-0.001616,-0.061952,-0.025852,-0.041427,0.015722,-0.032636,0.012678,-0.029335,-0.041698,0.001134,0.058911,-0.027985,0.063520,0.062227,0.063451,0.008441,-0.053186,0.038237,0.058160,...,0.026464,-0.036157,-0.023513,-0.061082,-0.020623,0.059865,0.060694,0.047033,-0.044069,-0.054903,-0.052173,-0.063486,0.038313,-0.061535,0.036823,0.054980,-0.059736,-0.062665,0.007966,0.008084,0.043164,0.033444,-0.049455,-0.023729,0.022630,-0.039991,0.063206,-0.024769,-0.005954,-0.057347,-0.016031,0.061994,0.044697,0.037051,0.026100,-0.059603,-0.000041,-0.036927,-0.048519,0.024104
4,0.061612,0.005061,0.005935,0.058571,-0.034820,-0.051958,-0.054881,-0.051389,0.020466,-0.062985,-0.062118,0.025811,-0.012073,0.056732,-0.061061,0.053391,0.044413,0.040475,0.034086,0.061892,0.046909,-0.001616,-0.061952,-0.025852,-0.041427,0.015722,-0.032636,0.012678,-0.029335,-0.041698,0.001134,0.058911,-0.027985,0.063520,0.062227,0.063451,0.008441,-0.053186,0.038237,0.058160,...,0.026464,-0.036157,-0.023513,-0.061082,-0.020623,0.059865,0.060694,0.047033,-0.044069,-0.054903,-0.052173,-0.063486,0.038313,-0.061535,0.036823,0.054980,-0.059736,-0.062665,0.007966,0.008084,0.043164,0.033444,-0.049455,-0.023729,0.022630,-0.039991,0.063206,-0.024769,-0.005954,-0

PCA

use PCA to reduce the dimensionality of the datsets

In [18]:
from sklearn.decomposition import PCA
pca = PCA(n_components=30)
principalComponents = pca.fit_transform(df)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component ' + str(i) for i in range (1, 31)])

In [19]:
principalDf

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,principal component 11,principal component 12,principal component 13,principal component 14,principal component 15,principal component 16,principal component 17,principal component 18,principal component 19,principal component 20,principal component 21,principal component 22,principal component 23,principal component 24,principal component 25,principal component 26,principal component 27,principal component 28,principal component 29,principal component 30
0,0.457397,0.079582,-0.064650,0.154000,-0.051931,-0.065731,0.144512,-0.008206,-0.063707,-0.129809,0.049156,-0.001380,0.098082,-0.016032,0.116505,0.222786,0.063976,0.054209,0.025591,0.109367,0.107742,0.042252,-0.074326,-0.019017,-0.004334,0.007400,0.030316,-0.041805,0.042604,-0.010564
1,-0.119754,-0.180509,-0.086904,0.045734,-0.066481,0.003221,-0.028784,0.155475,-0.050835,0.132253,0.081773,0.156263,0.005788,0.034023,0.102271,0.059769,0.059144,0.076589,0.007609,-0.030328,-0.034322,-0.000371,0.032079,0.041994,0.011804,0.037773,-0.073046,-0.082759,-0.006915,0.025038
2,-0.119754,-0.180507,-0.086904,0.045736,-0.066486,0.003220,-0.028780,0.155475,-0.050837,0.132256,0.081773,0.156267,0.005795,0.034023,0.102272,0.059779,0.059146,0.076588,0.007613,-0.030336,-0.034319,-0.000383,0.032091,0.041996,0.011798,0.037782,-0.073046,-0.082767,-0.006912,0.025034
3,-0.119754,-0.180506,-0.086903,0.045740,-0.066489,0.003220,-0.028780,0.155477,-0.050831,0.132257,0.081775,0.156262,0.005797,0.034022,0.102279,0.059781,0.059135,0.076582,0.007617,-0.030342,-0.034316,-0.000384,0.032086,0.041994,0.011788,0.037784,-0.073060,-0.082767,-0.006905,0.025029
4,-0.119754,-0.180506,-0.086901,0.045735,-0.066487,0.003220,-0.028780,0.155480,-0.050831,0.132253,0.081777,0.156275,0.005793,0.034026,0.102285,0.059782,0.059146,0.076593,0.007622,-0.030330,-0.034308,-0.000372,0.032086,0.041989,0.011806,0.037787,-0.073045,-0.082768,-0.006916,0.025038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119747,0.011129,-0.008916,0.021603,0.037218,0.082583,-0.020215,-0.108104,-0.022146,-0.174744,0.046273,0.086046,-0.092813,-0.012967,-0.003861,0.039911,0.011989,0.058866,-0.045105,0.110029,0.040964,-0.012471,-0.065183,-0.048784,-0.004911,-0.076402,-0.064735,-0.037579,-0.004866,0.017127,0.014999
119748,0.081500,0.017772,0.039567,0.029421,0.118863,0.031704,-0.119715,-0.027441,-0.136891,0.075590,0.079660,-0.079259,-0.008529,0.022139,0.012780,0.003614,0.039221,-0.049040,0.103763,0.021205,-0.017349,-0.068977,-0.055202,0.017110,-0.118578,-0.039631,-0.021054,-0.019840,0.011578,0.030478
119749,0.303651,0.128823,-0.081794,0.044084,0.082799,0.059774,-0.072841,-0.004992,-0.089783,0.028961,-0.051132,0.002673,0.055033,-0.053973,-0.028134,0.102661,0.000560,-0.026138,0.027409,0.081064,0.024627,0.017017,-0.088490,-0.053458,-0.062225,-0.026681,0.046793,-0.019948,0.020290,0.079971
119750,-0.184459,-0.164993,0.013894,0.138743,0.171132,-0.004630,0.225978,0.077402,-0.069028,-0.037848,0.043824,-0.040122,0.094004,0.081652,-0.057191,0.026414,0.136724,-0.076306,0.066192,-0.029467,0.054170,0.100218,0.011503,-0.023360,0.105171,0.066332,-0.083580,-0.075296,-0.031031,-0.017540


In [20]:
sum(pca.explained_variance_ratio_)

0.7678090270177174

In [ ]:
principalDf.to_csv('PCA_result_df.csv')

Test by xgboost

In [21]:
df_cross_1 = df_cross.join(principalDf)

In [22]:
df_cross_1 = df_cross_1.drop(columns = ['dtld_descr'])

In [ ]:
# df_cross_1.to_csv('df_cross_cnn_PCA.csv') 
# df_cross_1 = pd.read_csv('df_cross_cnn_PCA.csv')

In [26]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [24]:
X = df_cross_1.drop(columns = 'preference')
y = df_cross_1['preference']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [27]:
xgb1 = xgb.XGBRegressor()
parameters = {
              'objective':['reg:linear','reg:squarederror'],
              'learning_rate': [.03, 0.05], #so called `eta` value
              'max_depth': [5, 6, 7],
              'n_estimators': [10,20]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,y_train)

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  48 out of  48 | elapsed:  2.5min finished


[02:27:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=2, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=5,
             param_grid={'learning_rate': [0.03, 0.05], 'max_depth': [5, 6, 7],
                         'n_estimators': [10, 20],
    

In [28]:
print('best parameters: ', xgb_grid.best_params_)

best parameters:  {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 20, 'objective': 'reg:linear'}


In [30]:
y_train_pred = xgb_grid.predict(X_train)
y_pred = xgb_grid.predict(X_test)
mse_training = mean_squared_error(xgb_grid.predict(X_train), y_train)
mse_testing = mean_squared_error(xgb_grid.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.03580842196156337, testing mse: 0.03651029904889397
training rmse: 0.1892311337004653, testing rmse: 0.19107668368718872


Test by Random Forest

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [32]:
forest = RandomForestRegressor(random_state = 1, n_estimators = 10, min_samples_split = 2)
params = {
            'max_depth': [5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 5]
        }

rf = GridSearchCV(forest, params, cv = 3, verbose = 1, n_jobs = -1)
rf.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 16.2min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=None,
                                             oob_score=False, random_state=1,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-

In [33]:
print('best parameters: ', rf.best_params_)

best parameters:  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}


In [34]:
from sklearn.metrics import mean_squared_error as mse

y_train_pred = rf.predict(X_train)
y_pred = rf.predict(X_test)
mse_training = mse(rf.predict(X_train), y_train)
mse_testing = mse(rf.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.019589441483705474, testing mse: 0.021780560524517524
training rmse: 0.13996228593340948, testing rmse: 0.14758238554962283


Test by Linear Regression

In [35]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [36]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [38]:
import numpy as np

def mse(y_predicted, y_true):
  return np.mean(np.square(y_predicted - y_true))
mse_training = mse(lr.predict(X_train), y_train)
mse_testing = mse(lr.predict(X_test), y_test)
print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.023163369335312992, testing mse: 0.02327799105419951
training rmse: 0.1521951685675764, testing rmse: 0.1525712654932098
